In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# --- NBER U.S. Recession Dates ---
# Relevant NBER Recession: Feb 2020 - April 2020
RECESSION_START = datetime(2020, 2, 1)
RECESSION_END = datetime(2020, 4, 30)

def check_recession(date_str):
    """Checks if a given date falls within the defined recession period."""
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    return RECESSION_START <= date_obj <= RECESSION_END

# --- 1. Filtered Event Data: Only cases where CME projected a CUT (-1) ---
# New column: In_Recession (True if Blackout Start date is within the NBER recession)

event_data = [
    # [Announcement_Date, Blackout_Start_Date (Sat), CME_Prob_Percent, Expected_Change_Type (-1=Cut), In_Recession]
    
    # 2024-2025
    ["2025-10-29", "2025-10-18", 82.8, -1, check_recession("2025-10-18")],
    ["2025-09-17", "2025-09-06", 81.0, -1, check_recession("2025-09-06")],
    ["2024-12-18", "2024-12-07", 85.0, -1, check_recession("2024-12-07")],
    ["2024-11-07", "2024-10-26", 75.0, -1, check_recession("2024-10-26")],
    ["2024-09-18", "2024-09-07", 65.0, -1, check_recession("2024-09-07")],
    
    # Historical events 
    ["2020-03-16", "2020-03-07", 85.0, -1, check_recession("2020-03-07")], # Blackout start (3/7/2020) is in recession
    ["2020-03-03", "2020-02-22", 80.0, -1, check_recession("2020-02-22")], # Blackout start (2/22/2020) is BEFORE recession start (3/1/2020 is closer, but using 2/22 for consistency)
    ["2019-10-30", "2019-10-19", 90.0, -1, check_recession("2019-10-19")], # Not in recession
    ["2019-09-18", "2019-09-07", 95.0, -1, check_recession("2019-09-07")], # Not in recession
    ["2019-07-31", "2019-07-20", 80.0, -1, check_recession("2019-07-20")], # Not in recession
]

# For 2020-03-03 (Announcement), Blackout Start was 2020-02-22, which is technically BEFORE NBER's Feb 2020 start date.
# However, due to the nature of the economic stress, it's often grouped. Let's stick strictly to NBER.

# Re-running the check for clarity on historical dates:
# 2020-03-16: Blackout Start 2020-03-07 -> IN RECESSION (Feb 2020 start is used for the previous month)
# 2020-03-03: Blackout Start 2020-02-22 -> NBER starts Feb 2020, so this is IN RECESSION.
event_data[5][4] = True # 2020-03-16
event_data[6][4] = True # 2020-03-03
# The rest are clearly False.

# --- 2. Download QQQ Data ---
START_DATE = "2015-07-01" 
END_DATE = (datetime.now() + timedelta(days=365)).strftime("%Y-%m-%d") 

print(f"Downloading QQQ data from {START_DATE} to {END_DATE}...")
qqq_data = yf.download("QQQ", start=START_DATE, end=END_DATE, interval="1d", auto_adjust=True)
print("Download complete.")

# --- 3. Function to get non-trading days adjusted date (Robust version) ---

def get_market_date(df, target_date_str, is_open_price=False, offset=0):
    try:
        target_date_obj = datetime.strptime(target_date_str, "%Y-%m-%d")
    except ValueError:
        return None, None

    df_index_np = df.index.to_numpy()
    
    try:
        idx = df_index_np.searchsorted(np.datetime64(target_date_obj), side='left')
    except TypeError:
        return None, None

    if offset != 0:
        target_idx = idx + offset
        
        if 0 <= target_idx < len(df):
            actual_date = df.index[target_idx]
            price_type = 'Close'
            price_val = df.loc[actual_date, price_type]
            
            return float(price_val.item()), actual_date.strftime("%Y-%m-%d")
        else:
            return None, None
    
    if 0 <= idx < len(df):
        actual_date = df.index[idx]
        price_type = 'Open' if is_open_price else 'Close'
        
        if actual_date >= df.index.min():
            price_val = df.loc[actual_date, price_type]
            
            return float(price_val.item()), actual_date.strftime("%Y-%m-%d")
        else:
            return None, None
    else:
        return None, None


# --- 4. Process Events and Calculate Returns ---
results = []
for announcement_date_str, blackout_start_str, cme_prob, expected_change, in_recession in event_data:
    
    # 4.1. Blackout Period Performance (Blackout Start Open to T-1 Close)
    p_blackout_start, d_blackout_start = get_market_date(qqq_data, blackout_start_str, is_open_price=True)
    p_blackout_end, d_blackout_end = get_market_date(qqq_data, announcement_date_str, is_open_price=False, offset=-1)

    if p_blackout_start is not None and p_blackout_end is not None:
        blackout_return = (p_blackout_end / p_blackout_start - 1) * 100
    else:
        blackout_return = None
    
    # 4.2. Pre-Announcement Performance (T-3 Close to T-1 Close)
    p_t3_close, d_t3_close = get_market_date(qqq_data, announcement_date_str, is_open_price=False, offset=-3)
    p_t1_close, d_t1_close = p_blackout_end, d_blackout_end

    if p_t3_close is not None and p_t1_close is not None:
        pre_announcement_return = (p_t1_close / p_t3_close - 1) * 100
    else:
        pre_announcement_return = None

    # Append results
    results.append({
        'Decision_Date': announcement_date_str,
        'CME_Prob_Percent': cme_prob,
        'In_Recession': in_recession, # Added flag
        'Blackout_Start_Date': d_blackout_start,
        'Start_Open_Price': p_blackout_start,
        'T-1_Close_Price': p_blackout_end,
        'Blackout_Return': blackout_return,
        'Blackout_Win': 1 if blackout_return is not None and blackout_return > 0 else 0,
        'T-3_Date': d_t3_close,
        'T-3_Close_Price': p_t3_close,
        'Pre_Ann_Return': pre_announcement_return,
        'Pre_Ann_Win': 1 if pre_announcement_return is not None and pre_announcement_return > 0 else 0
    })

results_df = pd.DataFrame(results).dropna(subset=['Blackout_Return', 'Pre_Ann_Return'])

# --- 5. Prepare and Print Detailed Log (Including Recession Tag) ---
detailed_log_df = results_df[[
    'Decision_Date',
    'CME_Prob_Percent',
    'In_Recession', # Added to the log
    'Blackout_Start_Date',
    'Start_Open_Price',
    'T-1_Close_Price',
    'Blackout_Return',
    'T-3_Date',
    'T-3_Close_Price',
    'Pre_Ann_Return'
]].copy()

# Format floats for better readability
for col in ['Start_Open_Price', 'T-1_Close_Price', 'T-3_Close_Price']:
    detailed_log_df[col] = detailed_log_df[col].map('${:.2f}'.format)

detailed_log_df['Blackout_Return'] = detailed_log_df['Blackout_Return'].map('{:.2f}%'.format)
detailed_log_df['Pre_Ann_Return'] = detailed_log_df['Pre_Ann_Return'].map('{:.2f}%'.format)

# Rename columns for the final table
detailed_log_df.rename(columns={
    'CME_Prob_Percent': 'CME Prob % (Cut)',
    'In_Recession': 'Recession?',
    'Blackout_Start_Date': 'Blackout Start (Open)',
    'Start_Open_Price': 'QQQ Start Price',
    'T-1_Close_Price': 'QQQ End Price',
    'Blackout_Return': 'Blackout Return',
    'T-3_Date': 'T-3 Close Date',
    'T-3_Close_Price': 'T-3 Close Price',
    'Pre_Ann_Return': 'T-3 to T-1 Return'
}, inplace=True)


# --- 6. Separate and Print Summary Analysis Tables ---

# Binning logic
bins = np.arange(40, 105, 5)
labels = [f'{i}-{i+4}%' for i in bins[:-1]]
results_df['CME_Prob_Bin'] = pd.cut(results_df['CME_Prob_Percent'], bins=bins, labels=labels, right=False)

def create_summary(df, title):
    """Generates and formats the summary table for a filtered DataFrame."""
    
    # Group by the probability bin and calculate metrics
    summary_results = df.groupby('CME_Prob_Bin', observed=False).agg(
        Count=('CME_Prob_Percent', 'count'),
        Blackout_Win_Rate=('Blackout_Win', 'mean'),
        Blackout_Avg_Return=('Blackout_Return', 'mean'),
        Pre_Ann_Win_Rate=('Pre_Ann_Win', 'mean'),
        Pre_Ann_Avg_Return=('Pre_Ann_Return', 'mean')
    ).reset_index()

    # Format the output
    summary_results['Blackout_Win_Rate'] = (summary_results['Blackout_Win_Rate'] * 100).map('{:.2f}%'.format)
    summary_results['Blackout_Avg_Return'] = (summary_results['Blackout_Avg_Return']).map('{:.2f}%'.format)
    summary_results['Pre_Ann_Win_Rate'] = (summary_results['Pre_Ann_Win_Rate'] * 100).map('{:.2f}%'.format)
    summary_results['Pre_Ann_Avg_Return'] = (summary_results['Pre_Ann_Avg_Return']).map('{:.2f}%'.format)

    # Rename columns for clarity
    summary_results.rename(columns={
        'Pre_Ann_Win_Rate': 'T-3_to_T-1_Win_Rate',
        'Pre_Ann_Avg_Return': 'T-3_to_T-1_Avg_Return'
    }, inplace=True)
    
    print("\n" + "="*95)
    print(f"  📊 SUMMARY ANALYSIS: {title} (Grouped by CME FedWatch Probability)")
    print("="*95)
    print("Analysis Period 1: Blackout Period (Blackout Start Open to T-1 Close)")
    print("Analysis Period 2: Pre-Announcement (T-3 Close to T-1 Close)")
    print("-"*95)
    print(summary_results.to_markdown(index=False))
    print("\n" + "="*95)


# --- Apply Filters and Print Summaries ---
recession_df = results_df[results_df['In_Recession'] == True].copy()
non_recession_df = results_df[results_df['In_Recession'] == False].copy()

# --- Final Output Display ---
print("\n" + "="*120)
print("             🔍 DETAILED EVENT LOG: QQQ Performance When CME Market Expected A RATE CUT")
print("="*120)
print(detailed_log_df.to_markdown(index=False))

create_summary(recession_df, "Recession Cuts (Historical: Mar 2020)")
create_summary(non_recession_df, "Non-Recession Cuts (Historical: 2019 + Projected)")

[*********************100%***********************]  1 of 1 completed

Download complete.

             🔍 DETAILED EVENT LOG: QQQ Performance When CME Market Expected A RATE CUT
| Decision_Date   |   CME Prob % (Cut) | Recession?   | Blackout Start (Open)   | QQQ Start Price   | QQQ End Price   | Blackout Return   | T-3 Close Date   | T-3 Close Price   | T-3 to T-1 Return   |
|:----------------|-------------------:|:-------------|:------------------------|:------------------|:----------------|:------------------|:-----------------|:------------------|:--------------------|
| 2025-10-29      |               82.8 | False        | 2025-10-20              | $607.14           | $632.92         | 4.25%             | 2025-10-24       | $617.10           | 2.56%               |
| 2025-09-17      |               81   | False        | 2025-09-08              | $577.70           | $590.50         | 2.21%             | 2025-09-12       | $585.98           | 0.77%               |
| 2024-12-18      |               85   | False        | 2024-12-09              | $522.73

In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# --- 1. CONFIGURATION AND CORRECTED RECESSION LOGIC ---

# NBER U.S. Recession Start: February 2020. End: April 2020.
RECESSION_START_DATE = datetime(2020, 2, 1)
RECESSION_END_DATE = datetime(2020, 4, 30)

def check_recession(date_str):
    """
    Checks if a given date falls WITHIN the official NBER recession period.
    The check uses the Blackout Start Date for market context.
    """
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    return RECESSION_START_DATE <= date_obj <= RECESSION_END_DATE

# Event Data: CME projected a CUT (-1)
event_data = [
    # [Announcement_Date, Blackout_Start_Date (Sat), CME_Prob_Percent, Expected_Change_Type (-1=Cut)]
    
    # Projected future events 
    ["2025-10-29", "2025-10-18", 82.8, -1],
    ["2025-09-17", "2025-09-06", 81.0, -1],
    ["2024-12-18", "2024-12-07", 85.0, -1],
    ["2024-11-07", "2024-10-26", 75.0, -1],
    ["2024-09-18", "2024-09-07", 65.0, -1],
    
    # Historical events:
    ["2020-03-16", "2020-03-07", 85.0, -1], # CORRECTLY TAGGED AS IN RECESSION
    ["2020-03-03", "2020-02-22", 80.0, -1], # CORRECTLY TAGGED AS IN RECESSION
    ["2019-10-30", "2019-10-19", 90.0, -1],
    ["2019-09-18", "2019-09-07", 95.0, -1],
    ["2019-07-31", "2019-07-20", 80.0, -1],
]

# Add Recession Flag (using Blackout Start Date)
for item in event_data:
    item.append(check_recession(item[1])) 

# --- 2. DATA DOWNLOAD ---
START_DATE = "2015-07-01" 
END_DATE = (datetime.now() + timedelta(days=365)).strftime("%Y-%m-%d") 

print(f"Downloading QQQ data from {START_DATE} to {END_DATE}...")
qqq_data = yf.download("QQQ", start=START_DATE, end=END_DATE, interval="1d", auto_adjust=True)
print("Download complete.")

# --- 3. HELPER FUNCTION (Retained the correct T-X Open logic) ---

def get_market_date(df, target_date_str, is_open_price=False, offset=0):
    try:
        target_date_obj = datetime.strptime(target_date_str, "%Y-%m-%d")
    except ValueError:
        return None, None

    df_index_np = df.index.to_numpy()
    
    try:
        idx = df_index_np.searchsorted(np.datetime64(target_date_obj), side='left')
    except TypeError:
        return None, None

    if offset != 0:
        target_idx = idx + offset
        
        if 0 <= target_idx < len(df):
            actual_date = df.index[target_idx]
            price_type = 'Open' if is_open_price else 'Close' 
            price_val = df.loc[actual_date, price_type]
            
            return float(price_val.item()), actual_date.strftime("%Y-%m-%d")
        else:
            return None, None
    
    if 0 <= idx < len(df):
        actual_date = df.index[idx]
        price_type = 'Open' if is_open_price else 'Close'
        
        if actual_date >= df.index.min():
            price_val = df.loc[actual_date, price_type]
            
            return float(price_val.item()), actual_date.strftime("%Y-%m-%d")
        else:
            return None, None
    else:
        return None, None


# --- 4. PROCESS EVENTS AND CALCULATE RETURNS ---
results = []
for announcement_date_str, blackout_start_str, cme_prob, expected_change, in_recession in event_data:
    
    # --- Price Retrievals ---
    p_blackout_start, d_blackout_start = get_market_date(qqq_data, blackout_start_str, is_open_price=True)
    p_t1_close, d_t1_close = get_market_date(qqq_data, announcement_date_str, is_open_price=False, offset=-1)
    p_t2_open, d_t2_date = get_market_date(qqq_data, announcement_date_str, is_open_price=True, offset=-2) 
    p_t3_open, d_t3_date = get_market_date(qqq_data, announcement_date_str, is_open_price=True, offset=-3) 

    # --- Calculations ---
    blackout_return = (p_t1_close / p_blackout_start - 1) * 100 if p_blackout_start and p_t1_close else None
    t3_to_t1_return = (p_t1_close / p_t3_open - 1) * 100 if p_t3_open and p_t1_close else None
    t2_to_t1_return = (p_t1_close / p_t2_open - 1) * 100 if p_t2_open and p_t1_close else None

    # Append results
    results.append({
        'Decision_Date': announcement_date_str,
        'CME_Prob_Percent': cme_prob,
        'In_Recession': in_recession,
        'Blackout_Start_Date': d_blackout_start,
        'Start_Open_Price': p_blackout_start,
        'T-1_Close_Price': p_t1_close,
        'Blackout_Return': blackout_return,
        'Blackout_Win': 1 if blackout_return is not None and blackout_return > 0 else 0,
        'T-3_Date': d_t3_date,
        'T-3_Open_Price': p_t3_open, 
        'T3_to_T1_Return': t3_to_t1_return,
        'T3_to_T1_Win': 1 if t3_to_t1_return is not None and t3_to_t1_return > 0 else 0,
        'T-2_Date': d_t2_date,
        'T-2_Open_Price': p_t2_open, 
        'T2_to_T1_Return': t2_to_t1_return,
        'T2_to_T1_Win': 1 if t2_to_t1_return is not None and t2_to_t1_return > 0 else 0
    })

results_df = pd.DataFrame(results).dropna(subset=['Blackout_Return', 'T3_to_T1_Return', 'T2_to_T1_Return'])

# --- 5. PREPARE DETAILED LOG ---
detailed_log_df = results_df[[
    'Decision_Date',
    'CME_Prob_Percent',
    'In_Recession', 
    'Blackout_Start_Date',
    'Start_Open_Price',
    'T-1_Close_Price',
    'Blackout_Return',
    'T-3_Date',
    'T-3_Open_Price',
    'T3_to_T1_Return',
    'T-2_Date',
    'T-2_Open_Price',
    'T2_to_T1_Return'
]].copy()

# Formatting
for col in ['Start_Open_Price', 'T-1_Close_Price', 'T-3_Open_Price', 'T-2_Open_Price']:
    detailed_log_df[col] = detailed_log_df[col].map('${:.2f}'.format)

for col in ['Blackout_Return', 'T3_to_T1_Return', 'T2_to_T1_Return']:
    detailed_log_df[col] = detailed_log_df[col].map('{:.2f}%'.format)

# Renaming columns for final display
detailed_log_df.rename(columns={
    'CME_Prob_Percent': 'CME Prob % (Cut)',
    'In_Recession': 'Recession?',
    'Blackout_Start_Date': 'Blackout Start (Open)',
    'Start_Open_Price': 'QQQ Blackout Start Price',
    'T-1_Close_Price': 'QQQ T-1 Close Price',
    'Blackout_Return': 'Blackout Return',
    'T-3_Date': 'T-3 Start Date',
    'T-3_Open_Price': 'QQQ T-3 Open Price',
    'T3_to_T1_Return': 'T-3 Open to T-1 Close Return',
    'T-2_Date': 'T-2 Start Date',
    'T-2_Open_Price': 'QQQ T-2 Open Price',
    'T2_to_T1_Return': 'T-2 Open to T-1 Close Return'
}, inplace=True)


# --- 6. SEPARATE AND PRINT SUMMARY ANALYSIS TABLES ---

# Binning logic
bins = np.arange(40, 105, 5)
labels = [f'{i}-{i+4}%' for i in bins[:-1]]
results_df['CME_Prob_Bin'] = pd.cut(results_df['CME_Prob_Percent'], bins=bins, labels=labels, right=False)

def create_summary(df, title):
    """Generates and formats the summary table for a filtered DataFrame."""
    
    summary_results = df.groupby('CME_Prob_Bin', observed=False).agg(
        Count=('CME_Prob_Percent', 'count'),
        # Blackout Metrics
        Blackout_Win_Rate=('Blackout_Win', 'mean'),
        Blackout_Avg_Return=('Blackout_Return', 'mean'),
        # T-3 to T-1 Metrics
        T3_to_T1_Win_Rate=('T3_to_T1_Win', 'mean'),
        T3_to_T1_Avg_Return=('T3_to_T1_Return', 'mean'),
        # T-2 to T-1 Metrics
        T2_to_T1_Win_Rate=('T2_to_T1_Win', 'mean'),
        T2_to_T1_Avg_Return=('T2_to_T1_Return', 'mean')
    ).reset_index()

    # Formatting
    for col_prefix in ['Blackout', 'T3_to_T1', 'T2_to_T1']:
        summary_results[f'{col_prefix}_Win_Rate'] = (summary_results[f'{col_prefix}_Win_Rate'] * 100).map('{:.2f}%'.format)
        summary_results[f'{col_prefix}_Avg_Return'] = (summary_results[f'{col_prefix}_Avg_Return']).map('{:.2f}%'.format)

    print("\n" + "="*120)
    print(f"  📊 SUMMARY ANALYSIS: {title} (Grouped by CME FedWatch Probability)")
    print("="*120)
    print("Period 1: Blackout Start (Open) to T-1 Close")
    print("Period 2: T-3 (Open) to T-1 Close")
    print("Period 3: T-2 (Open) to T-1 Close")
    print("-"*120)
    print(summary_results.to_markdown(index=False))
    print("\n" + "="*120)


# --- FINAL OUTPUT DISPLAY ---
recession_df = results_df[results_df['In_Recession'] == True].copy()
non_recession_df = results_df[results_df['In_Recession'] == False].copy()

print("\n" + "="*160)
print("             🔍 DETAILED EVENT LOG: QQQ Performance When CME Market Expected A RATE CUT")
print("="*160)
print(detailed_log_df.to_markdown(index=False))

create_summary(recession_df, "Recession Cuts (Historical: Mar 2020)")
create_summary(non_recession_df, "Non-Recession Cuts (Historical: 2019 + Projected 2024/2025)")

[*********************100%***********************]  1 of 1 completed

Download complete.

             🔍 DETAILED EVENT LOG: QQQ Performance When CME Market Expected A RATE CUT
| Decision_Date   |   CME Prob % (Cut) | Recession?   | Blackout Start (Open)   | QQQ Blackout Start Price   | QQQ T-1 Close Price   | Blackout Return   | T-3 Start Date   | QQQ T-3 Open Price   | T-3 Open to T-1 Close Return   | T-2 Start Date   | QQQ T-2 Open Price   | T-2 Open to T-1 Close Return   |
|:----------------|-------------------:|:-------------|:------------------------|:---------------------------|:----------------------|:------------------|:-----------------|:---------------------|:-------------------------------|:-----------------|:---------------------|:-------------------------------|
| 2025-10-29      |               82.8 | False        | 2025-10-20              | $607.14                    | $632.92               | 4.25%             | 2025-10-24       | $615.99              | 2.75%                          | 2025-10-27       | $624.52              | 1.35%      